In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
os.chdir('/content/drive/MyDrive/pk추가')
user_df= pd.read_csv("./선곡표종합.csv", encoding = 'utf-8')
songDB= pd.read_csv("./preprocessed_songDB.csv", encoding= 'utf-8')
tbn= pd.read_csv("./tbn_종합.csv", encoding= 'utf-8')
prime = pd.read_excel("./tbn_allinone_all_final_url -backup_마지막.xlsx", usecols= ['pk', 'title'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# # 작게 돌리고 싶을 때 (sample 0.01)
# user_df_sample = user_df.sample(n=int(len(user_df)*0.01))
# tbn_sample = tbn.sample(n=int(len(tbn)*0.01))
# user_df_org = user_df.copy()
# tbn_org = tbn.copy()
# user_df = user_df_sample
# tbn = tbn_sample

In [ ]:
# FM 모델에서 쓰는 데이터 저장할 path 설정
path = '/content/drive/MyDrive/temp/'

##전처리 함수

In [ ]:
import numpy as np
import re
import datetime as dt
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
from dateutil.parser import parse
from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from collections import Counter 
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec

# FM
import tensorflow as tf
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import BinaryAccuracy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from sklearn import datasets

#! pip install xlearn
#! pip install gensim
#전처리
def preprocess_song(df):
  df = df.apply(lambda x: re.sub('\([^)]*\)',"",str(x)))
  df = df.apply(lambda x: x.replace("'", ""))
  df = df.apply(lambda x: x.lower())
  df = df.apply(lambda x: re.sub('[^A-Za-z0-9가-힣]', ' ', x))
  df = df.apply(lambda x: re.sub(" +", " ", x))
  df = df.apply(lambda x: x.strip())
  return df

#년도 전처리
def preprocess_year(result):
  result['year'] = list(map(str, result['year']))
  result['month'] = list(map(str, result['month']))
  result['date'] = list(map(str, result['date']))
  return result

#방송 시작일 전처리
def preprocess_airdate(result):
  result['air_date']= result['year'] + "-" + result['month'] + '-'+ result['date']
  result['air_date'] = list(map(str, result['air_date']))
  result['air_date'] = pd.to_datetime(result['air_date'])
  # result['air_date'] = result['air_date'].apply(lambda x : parse(x))
  return result

#장르 전처리
def preprocess_genre(result):
    result['genre'] = result['genre'].apply(lambda x: sp(x)) 
    result = result.replace({'genre' : change_value})  
    result['genre'] = result['genre'].apply(lambda x : x.strip())
    return result


# 프로그램별 빈도수 구하기
def data_rating(result):
    df = pd.DataFrame(data = result.groupby('ProgramName')['song_id'].value_counts())
    df.rename(columns = {'song_id': 'rating'}, inplace = True)
    dat_ = pd.merge(result, df, on=['ProgramName','song_id'], how = 'inner')
    return dat_

# MinMaxScaler를 통해 Scaling   
def minmax_scaler(dat_):
    uniq=dat_['ProgramName'].unique()
    n = len(uniq)
    for i in range(n):
        X = dat_.loc[dat_['ProgramName'] == uniq[i] , ['rating']]
        MinMaxScalers = MinMaxScaler()
        df_MinMaxScalers=MinMaxScalers.fit_transform(X)
        dat_.loc[dat_['ProgramName'] == uniq[i] , ['rating']] = df_MinMaxScalers
    return dat_

#song_id 별 likes sacling
def likes_scaling(dat_):
    X = np.reshape([dat_['likes']], (-1, 1))
    MinMaxScalers = MinMaxScaler()
    MinMaxScalers.fit(X)
    df_MinMaxScalers = MinMaxScalers.transform(X)
    dat_['likes'] = df_MinMaxScalers
    return dat_

   
#방송일 전처리
def parse_(x):
    return  parse(x)
#장르 전처리
def sp(x):
    return x.split(',')[0]

change_value = {"포크" : 'Folk', '랩/힙합' : 'Hip Hop', '댄스' : 'Dance', '록/메탈' : 'Rock', 'R&B/Soul' : 'R&B', '성인가요' : 'adult', '인디음악' : 'Indie'
               , '일렉트로니카' : 'Electronica', '재즈' : 'Jazz', '포크/블루스' : 'Folk', '뉴에이지' : 'Newage', '월드뮤직' : 'Worldmusic', '국내영화' : 'Domestic movie'
                , '국외영화' : 'Foreign movie', '국내드라마' : 'Domestic drama', '컨트리' : 'Country', '클래식' : 'Classic', '블루스' : 'Blues'
                , '국악' : 'Korean classical', '애니메이션/웹툰' : 'Animation', '키즈' : 'Kids', "국내뮤지컬" : 'Domestic musicals'
                , '국외뮤지컬' : 'Foreign musicals', '게임' : 'Game', '국외드라마' : 'Foreign drama', '불교' : 'Buddhism', '뮤직테라피' : 'Music Therapy', '발라드' : 'Ballade', '-' : 'None' }

## target(y) 변수를 만드는 것과 연관이 있어 일단 보류
#빈도수 전처리
def gb_rating(x):
    if x <= 0.001: return 1
    if ((x >  0.001) & (x <= 0.05)) : return 2
    if ((x >  0.05) & (x <= 0.3)) : return 3
    if ((x >  0.3) & (x <= 0.6)) : return 4
    return 5

#좋아요 전처리                             
def gb_likes(x):
    if x <= 0.02: return 1
    if ((x >  0.02) & (x <= 0.05)) : return 2
    if ((x >  0.05) & (x <= 0.1)) : return 3
    if ((x >  0.1) & (x <= 0.3)) : return 4
    return 5

#빈도수와 likes 수를 점수화 시키키
def rat_like(dat_):
    dat_['rating'] = round(dat_['rating'],4)
    dat_['rating'] = dat_['rating'].apply(lambda x: gb_rating(x))
    dat_['likes'] = dat_['likes'].apply(lambda x: gb_likes(x))
    return dat_

################# 22.01.17 추가 함수############## start
def pre_w0(x):
  for j in x.split('/'): 
    if str(j) in '맑음구름조금':
      return int(1) 
def pre_w1(x):
  for j in x.split('/'): 
    if str(j) in '흐림구름많음':
      return int(1) 
def pre_w2(x):
  for j in x.split('/'): 
    if str(j) in '안개':
      return int(1) 
def pre_w3(x):
  for j in x.split('/'): 
    if str(j) in '비진눈깨비이슬비소나기':
      return int(1) 
def pre_w4(x):
  for j in x.split('/'): 
    if str(j) in '눈우박소낙눈':
      return int(1) 

def pre_w5(x):
  for j in x.split('/'): 
    if str(j) in '천둥번개':
      return int(1) 
def pre_w6(x):
  for j in x.split('/'): 
    if str(j) in '황사':
      return int(1) 

# 날씨 전처리
def preprocess_weather(result):
  df2 = pd.DataFrame(index=range(0,len(result)), columns=['weather_clear', 'weather_cloud','weather_fog','weather_rain','weather_snow','weather_thunder','weather_disaster'])
  
  df2['weather_clear']=result['weather'].apply(lambda x:pre_w0(x))
  df2['weather_cloud']=result['weather'].apply(lambda x:pre_w1(x))
  df2['weather_fog']=result['weather'].apply(lambda x:pre_w2(x))
  df2['weather_rain']=result['weather'].apply(lambda x:pre_w3(x))
  df2['weather_snow']=result['weather'].apply(lambda x:pre_w4(x))
  df2['weather_thunder']=result['weather'].apply(lambda x:pre_w5(x))
  df2['weather_disaster']=result['weather'].apply(lambda x:pre_w6(x))
  df2=df2.fillna(0)
  df2 = df2.astype('int')
  result=pd.concat([result, df2],axis=1)
  result=result.fillna(0)
  return result


################# 22.01.17 추가 함수############## end


# #온도를 점수화 시켜줌
def temp_bucketize(dat_): # 연평균, 최저기온, 최고기온W
  x = dat_['평균기온']
  y = dat_['최저기온']
  z = dat_['최고기온']

  if (y <= -12) : return '0' # 한파
  if (z >= 33) : return '8' # 폭염
  if (x <= -6) : return '1'
  if ((x >  -6) & (x <= 0)) : return '2'
  if ((x >  0) & (x <= 6)) : return '3'
  if ((x >  6) & (x <= 12)) : return '4'
  if ((x >  12) & (x <= 18)) : return '5'
  if ((x >  18) & (x <= 24)) : return '6'
  if (x > 24) : return '7' 

# 강수량 : 시간당 강수 기준 (기상청)
def rain_bucketize(x):
  if x ==0 : return '0'
  if ((x >  0) & (x <= 3)) : return '1'
  if ((x >  3) & (x <= 15)) : return '2'
  if ((x >  15) & (x <= 30)) : return '3'
  return '4'

def temp_diff_bucketize(x):
  if x <= 2.5 : return '0'
  if ((x >  2.5) & (x <= 5)) : return '1'
  if ((x >  5) & (x <= 7.5)) : return '2'
  if ((x >  7.5) & (x <= 10)) : return '3' # 일교차 10 정도가 크다고 판단
  if ((x >  10) & (x <= 12.5)) : return '4'
  if ((x >  12.5) & (x <= 15)) : return '5'
  return '6'
  
# #온도를 점수화 시켜줌
# def temp_bucketize(x):
#     if (x <= -5) : return '-10'
#     if ((x >  -5) & (x <= 0)) : return '-5'
#     if ((x >  0) & (x <= 5)) : return '0'
#     if ((x >  5) & (x <= 10)) : return '5'
#     if ((x >  10) & (x <= 15)) : return '10'
#     if ((x >  15) & (x <= 20)) : return '15'
#     return '20'


# def rain_bucketize(x):
#   if x ==0 : return '0'
#   if ((x >  0) & (x <= 1)) : return '1'
#   if ((x >  1) & (x <= 3)) : return '2'
#   if ((x >  3) & (x <= 5)) : return '3'
#   return '4'

# def temp_diff_bucketize(x):
#   if x <=2.5 : return '0'
#   if ((x >  2.5) & (x <= 5)) : return '1'
#   if ((x >  5) & (x <= 7.5)) : return '2'
#   if ((x >  7.5) & (x <= 10)) : return '3'
#   if ((x >  10) & (x <= 12.5)) : return '4'
#   if ((x >  12.5) & (x <= 15)) : return '5'
#   return '6'
                             
                                      

##분석을 하기 위해서 dict로 저장 -> 피쳐맵을 맵핑하는데 사용
def meta2dic(dat_, col):
  lst_ = dat_[col].unique()
  col_index = {}
  for idx, col in enumerate(lst_):
      col_index[col] = idx + 1
  return col_index



##train, test데이터 분리
# def train_test_split(dat_):
#     train_df = dat_[(dat_['ProgramName'] == 0) & (dat_['air_date'] <= dat_.groupby('ProgramName')['air_date'].quantile(q = 0.7, interpolation = 'nearest')[0])]
#     for i in range(1, len(dat_['ProgramName'].unique())):
#         df_gb = dat_[(dat_['ProgramName'] == i) & (dat_['air_date'] <= dat_.groupby('ProgramName')['air_date'].quantile(q = 0.7, interpolation = 'nearest')[i])]
#         train_df = pd.concat([train_df, df_gb])
    
#     test_df = dat_[(dat_['ProgramName'] == 0) & (dat_['air_date'] > dat_.groupby('ProgramName')['air_date'].quantile(q = 0.7, interpolation = 'nearest')[0])]
#     for i in range(1, len(dat_['ProgramName'].unique())):
#         df_gb2 = dat_[(dat_['ProgramName'] == i) & (dat_['air_date'] > dat_.groupby('ProgramName')['air_date'].quantile(q = 0.7, interpolation = 'nearest')[i])]
#         test_df = pd.concat([test_df, df_gb2])
#     return train_df, test_df

def train_test_split(dat_):
  data = dat_
  target = dat_['ProgramName']

  # train_test_split
  # train_df, test_df, train_y, test_y = model_selection.train_test_split(data, target, test_size=0.3, shuffle=True, stratify=target, random_state=34)
  train_df, test_df, train_y, test_y = model_selection.train_test_split(data, target, test_size=0.3, shuffle=True, random_state=34)

  return train_df, test_df

        
def col_dict(train_df):
    col_len_dict = {'ProgramName': len(train_df['ProgramName'].unique()),
                'rating': len(train_df['rating'].unique()),
                'likes': len(train_df['likes'].unique()), 
                'genre': len(train_df['genre'].unique()), 
                # 'weather': len(train_df['weather'].unique()), 
                'weather_clear': len(train_df['weather_clear'].unique()), 
                'weather_cloud': len(train_df['weather_cloud'].unique()), 
                'weather_fog': len(train_df['weather_fog'].unique()), 
                'weather_rain': len(train_df['weather_rain'].unique()), 
                'weather_snow': len(train_df['weather_snow'].unique()), 
                'weather_thunder': len(train_df['weather_thunder'].unique()), 
                'weather_disaster': len(train_df['weather_disaster'].unique()), 
                'avg_temp': len(train_df['avg_temp'].unique()),
                'release': len(train_df['release'].unique()),
                'rain': len(train_df['rain'].unique()),
                'temp_diff': len(train_df['temp_diff'].unique()),
                'month': len(train_df['month'].unique()),
                'song_id': len(train_df['song_id'].unique()),
                'day': len(train_df['day'].unique())
                }
    col_accum_index_dict = {}
    cumulative = 0
    for key, value in col_len_dict.items():
        col_accum_index_dict[key] = cumulative
        cumulative = cumulative + value
    return col_accum_index_dict


# ##모델링 파트
# class FactorizationMachine():
#     """
#     This Class is implementation of this paper : https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf
#     Just a example of FM Algorithm, not for production.
#     -----
#     Only simple methods are available.
#     e.g 1 : batch training, adagrad optimizer, parallel training are not supported.
#     e.g 2 : simple optimizer Stochastic Gradient Descent with L2 Regularization.
#     e.g 3 : using titanic dataset on local memory.
#     """
#     def __init__(self, k, lr, l2_reg, l2_lambda, epoch, early_stop_window, train_data, valid_data):
#         """
#         :param k: number of latent vector
#         :param lr: learning rate
#         :param l2_reg: bool parameter for L2 regularization
#         :param l2_lambda: lambda of L2 regularization
#         :param epoch: training epoch
#         :param train_data: path of train data
#         :param valid_data: path of valid data
#         """
#         self._k = k
#         self._lr = lr
#         self._l2_reg = l2_reg
#         self._l2_lambda = l2_lambda
#         self._epoch = epoch
#         self._early_stop_window = early_stop_window
#         self._train_file_path = train_data
#         self._valid_file_path = valid_data
#         self._valid_loss_list = []

#     def _load_dataset(self):
#         """
#         1. load dataset to memory from train/valid path
#         2. find max index in dataset for w's vector size
#         """
#         # load data
#         train_file = open(self._train_file_path, 'r')
#         valid_file = open(self._valid_file_path, 'r')
#         self._train_data = train_file.read().split('\n')
#         self._valid_data = valid_file.read().split('\n')
#         train_file.close()
#         valid_file.close()

#         # find max index
#         self.feature_max_index = 0
#         print("Start to init FM vectors.")
#         for row in self._train_data:
#             for element in row.split(" ")[1:]:
#                 index = int(element.split(":")[0])
#                 if self.feature_max_index < index:
#                     self.feature_max_index = index

#         for row in self._valid_data:
#             for element in row.split(" ")[1:]:
#                 index = int(element.split(":")[0])
#                 if self.feature_max_index < index:
#                     self.feature_max_index = index

#         # init FM vectors
#         self._init_vectors()
#         print("Finish init FM vectors.")

#     def _init_vectors(self):
#         """
#         1. initialize FM vectors
#         2. Conduct naive transformation libsvm format txt data to numpy training sample.
#         """
#         self.w = np.random.randn(self.feature_max_index+1)
#         self.v = np.random.randn(self.feature_max_index+1, self._k)
#         self.train_x_data = []
#         self.train_y_data = np.zeros((len(self._train_data)-1))
#         self.valid_x_data = []
#         self.valid_y_data = np.zeros((len(self._valid_data)-1))

#         # make numpy dataset
#         for n, row in enumerate(self._train_data):
#             element = row.split(" ")
#             if len(element) > 1:
#               self.train_y_data[n] = int(element[0])
#               self.train_x_data.append([np.array([int(pair.split(":")[0]) for pair in element[1:]]),
#                                         np.array([int(pair.split(":")[1]) for pair in element[1:]])])

#         for n, row in enumerate(self._valid_data):
#             element = row.split(" ")
#             if len(element) > 1:
#               self.valid_y_data[n] = int(element[0])
#               self.valid_x_data.append([np.array([int(pair.split(":")[0]) for pair in element[1:]]),
#                                         np.array([int(pair.split(":")[1]) for pair in element[1:]])])

#     def train(self):
#         """
#         Train FM model by Gradient Descent with L2 regularization
#         """
#         self._load_dataset()
#         for epoch_num in range(1, self._epoch):
#             train_y_hat = self.predict(data=self.train_x_data)
#             valid_y_hat = self.predict(data=self.valid_x_data)
#             train_loss = self._get_loss(y_data=self.train_y_data, y_hat=train_y_hat)
#             valid_loss = self._get_loss(y_data=self.valid_y_data, y_hat=valid_y_hat)
#             train_auc = roc_auc_score(self.train_y_data, train_y_hat)
#             valid_auc = roc_auc_score(self.valid_y_data, valid_y_hat)
#             self._print_learning_info(epoch=epoch_num, train_loss=train_loss, valid_loss=valid_loss,
#                                       train_auc=train_auc, valid_auc=valid_auc)
#             if self._check_early_stop(valid_loss=valid_loss):
#                 print("Early stop at epoch:", epoch_num)
#                 return 0

#             self._stochastic_gradient_descent(self.train_x_data, self.train_y_data)

#     def predict(self, data):
#         """
#         Implementation of FM model's equation on O(kmd)
#         -----
#         Numpy array shape : (n, [index of md], [value of md])
#         md : none-zero feature
#         """
#         num_data = len(data)
#         scores = np.zeros(num_data)
#         for n in range(num_data):
#             feat_idx = data[n][0]
#             val = data[n][1]

#             # linear feature score
#             linear_feature_score = np.sum(self.w[feat_idx] * val)

#             # factorized feature score
#             vx = self.v[feat_idx] * (val.reshape(-1, 1))
#             cross_sum = np.sum(vx, axis=0)
#             square_sum = np.sum(vx * vx, axis=0)
#             cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

#             # Model's equation
#             scores[n] = linear_feature_score + cross_feature_score

#         # Sigmoid transformation for binary classification
#         scores = 1.0 / (1.0 + np.exp(-scores))
#         return scores

#     def _get_loss(self, y_data, y_hat):
#         """
#         Calculate loss with L2 regularization (two type of coeficient - w,v)
#         """
#         l2_norm = 0
#         if self._l2_reg:
#             w_norm = np.sqrt(np.sum(np.square(self.w)))
#             v_norm = np.sqrt(np.sum(np.square(self.v)))
#             l2_norm = self._l2_lambda * (w_norm + v_norm)
#         return -1 * np.sum( (y_data * np.log(y_hat)) + ((1 - y_data) * np.log(1 - y_hat)) ) + l2_norm

#     def _check_early_stop(self, valid_loss):
#         self._valid_loss_list.append(valid_loss)
#         if len(self._valid_loss_list) > 5:
#             prev_loss = self._valid_loss_list[len(self._valid_loss_list) - self._early_stop_window]
#             curr_loss = valid_loss
#             if prev_loss < curr_loss:
#                 return True
#         return False

#     def _print_learning_info(self, epoch, train_loss, valid_loss, train_auc, valid_auc):
#         print("epoch:", epoch, "||", "train_loss:", train_loss, "||", "valid_loss:", valid_loss,
#               "||", "Train AUC:", train_auc, "||", "Test AUC:", valid_auc)


#     def _stochastic_gradient_descent(self, x_data, y_data):
#         """
#         Update each coefs (w, v) by Gradient Descent
#         """
#         for data, y in zip(x_data, y_data):
#             feat_idx = data[0]
#             val = data[1]
#             vx = self.v[feat_idx] * (val.reshape(-1, 1))

#             # linear feature score
#             linear_feature_score = np.sum(self.w[feat_idx] * val)

#             # factorized feature score
#             vx = self.v[feat_idx] * (val.reshape(-1, 1))
#             cross_sum = np.sum(vx, axis=0)
#             square_sum = np.sum(vx * vx, axis=0)
#             cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

#             # Model's equation
#             score = linear_feature_score + cross_feature_score
#             y_hat = 1.0 / (1.0 + np.exp(-score))
#             cost = y_hat - y

#             if self._l2_reg:
#                 self.w[feat_idx] = self.w[feat_idx] - cost * self._lr * (val + self._l2_lambda * self.w[feat_idx])
#                 self.v[feat_idx] = self.v[feat_idx] - cost * self._lr * ((sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1)))) + self._l2_lambda * self.v[feat_idx])
#             else:
#                 self.w[feat_idx] = self.w[feat_idx] - cost * self._lr * val
#                 self.v[feat_idx] = self.v[feat_idx] - cost * self._lr * (sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1))))
                             

#프로그램별 노래를 rating의 오름차순으로 정렬
def get_music_list_sort_by_rating(x):
    return x.sort_values(['rating'])['song_id'].tolist()

#최근 틀었던 노래
def get_recent_song_list_sort_by_time(x, k):
        return x.sort_values(['air_date'])['song_id'].tolist()[-k:]


##인기많은 곡을 중복 제거해서 뽑기
def get_meta_popular_list_candidate(x, k):
    song_id_list = x.sort_values(by=['mean'], ascending=False)['song_id'].tolist()
    song_id_list = list(OrderedDict.fromkeys(song_id_list ))

    return song_id_list[:k]

def most_frequent(x): 
    occurence_count = Counter(x) 
    return occurence_count.most_common(1)[0][0] 

################# 22.01.17 추가 함수############## start

def get_meta_popular_list_candidate_weather(x, k):
  total_list=[]
  for weather in ['weather_clear','weather_cloud','weather_fog','weather_rain','weather_snow','weather_thunder','weather_disaster']:
    song_id_list = x[x[weather]==1].sort_values(by=['mean'], ascending=False)['song_id'].tolist()
    song_id_list = list(OrderedDict.fromkeys(song_id_list ))
    total_list.append(song_id_list[:k])

  return total_list




### Feature Mapping - Profiling 피처맵 맵핑
def featre_mapping(dat_, df):
    # weather_index = meta2dic(dat_, 'weather')
    # avg_temp_index = meta2dic(dat_, 'avg_temp')
    # rain_index =  meta2dic(dat_, 'rain')
    # temp_diff_index =  meta2dic(dat_, 'temp_diff')
    # month_index =  meta2dic(dat_, 'month')
    day_index =  meta2dic(dat_, 'day')
    genre_index = meta2dic(dat_, 'genre')
    year_index = meta2dic(dat_, 'release')

    df = df[['ProgramName', 'rating', 'likes','weather_clear','weather_cloud','weather_fog','weather_rain','weather_snow','weather_thunder','weather_disaster', 'avg_temp', 'genre', 'release', 'song_id', 'rain', 'temp_diff', 'month', 'day']]
    df['genre'] = df['genre'].apply(lambda x: genre_index[x])
    df['release'] = df['release'].apply(lambda x: year_index[x])
    # df['weather'] = df['weather'].apply(lambda x: weather_index[x])
    # df['avg_temp'] = df['avg_temp'].apply(lambda x: avg_temp_index[x])
    # df['rain'] = df['rain'].apply(lambda x: rain_index[x])
    # df['temp_diff'] = df['temp_diff'].apply(lambda x: temp_diff_index[x])
    # df['month'] = df['month'].apply(lambda x: month_index[x])
    df['day'] = df['day'].apply(lambda x: day_index[x])
    df['y'] = df['rating'].apply(lambda x : 1 if x >= 2 else 0)
    return df

    ##추천결과 평가
def eval_mapping(dat_):
    # weather_index = meta2dic(dat_, 'weather')
    # avg_temp_index = meta2dic(dat_, 'avg_temp')
    # rain_index =  meta2dic(dat_, 'rain')
    # temp_diff_index =  meta2dic(dat_, 'temp_diff')
    # month_index =  meta2dic(dat_, 'month')
    day_index =  meta2dic(dat_, 'day')
    genre_index = meta2dic(dat_, 'genre')
    year_index = meta2dic(dat_, 'release')

    program_df = dat_[['ProgramName', 'weather_clear','weather_cloud','weather_fog','weather_rain','weather_snow','weather_thunder','weather_disaster','avg_temp', 'rain', 'temp_diff', 'month', 'day']].copy()
    # program_df['weather'] = dat_['weather'].apply(lambda x: weather_index[x])
    # program_df['avg_temp'] = dat_['avg_temp'].apply(lambda x: avg_temp_index[x])
    # program_df['rain'] = dat_['rain'].apply(lambda x: rain_index[x])
    # program_df['temp_diff'] = dat_['temp_diff'].apply(lambda x: temp_diff_index[x])
    # program_df['month'] = dat_['month'].apply(lambda x: month_index[x])
    program_df['day'] = dat_['day'].apply(lambda x: day_index[x])

    song_df = dat_[['song_id','release', 'genre']].copy()
    song_df['release'] = song_df['release'].apply(lambda x: year_index[x])
    song_df['genre'] = song_df['genre'].apply(lambda x: genre_index[x])

    return program_df, song_df

################# 22.01.17~18 추가 함수############## end




#최근 틀었던 노래와 유사한곡을 추출하기 위한 함수.
def get_similar_items(item2item, x, k):
    similar_items = []
    for song_id in x:
        if song_id in item2item:
            similar_items.append(item2item[song_id][:k])
    return [item for items in similar_items for item in items]

###예측값 추출
def make_libsvm_row(col_accum_index_dict, program_index_dict, song_index_dict, uid, sid):
  user_id = str(col_accum_index_dict['ProgramName'] + uid)
  weather_clear = str(col_accum_index_dict['weather_clear'] + program_index_dict['weather_clear'][uid])
  weather_cloud = str(col_accum_index_dict['weather_cloud'] + program_index_dict['weather_cloud'][uid])
  weather_fog = str(col_accum_index_dict['weather_fog'] + program_index_dict['weather_fog'][uid])
  weather_rain = str(col_accum_index_dict['weather_rain'] + program_index_dict['weather_rain'][uid])
  weather_snow = str(col_accum_index_dict['weather_snow'] + program_index_dict['weather_snow'][uid])
  weather_thunder = str(col_accum_index_dict['weather_thunder'] + program_index_dict['weather_thunder'][uid])
  weather_disaster = str(col_accum_index_dict['weather_disaster'] + program_index_dict['weather_disaster'][uid])
  avg_temp = str(col_accum_index_dict['avg_temp'] + int(program_index_dict['avg_temp'][uid]))
  rain = str(col_accum_index_dict['rain'] + int(program_index_dict['rain'][uid]))
  temp_diff = str(col_accum_index_dict['temp_diff'] + int(program_index_dict['temp_diff'][uid]))
  month = str(col_accum_index_dict['month'] + int(program_index_dict['month'][uid]) - 1)
  day = str(col_accum_index_dict['day'] + program_index_dict['day'][uid] - 1)
  # song_id = str(col_accum_index_dict['song_id'] + int(sid) - 1)    #???????????????
  release = str(col_accum_index_dict['release'] + song_index_dict['release'][int(sid)] - 1)
  genre = str(col_accum_index_dict['genre'] + song_index_dict['genre'][int(sid)] - 1)   
  return " ".join([user_id, weather_clear,weather_cloud,weather_fog,weather_rain,weather_snow,weather_thunder,weather_disaster, release, genre, avg_temp, rain, temp_diff, month, day ])

############################ 22.01.17 일부 수정 ###########################################
def recomend_list(recommendations, dat_, program_id):
    recomend = list(map(int, list(recommendations[program_id])))
    reco = dat_[dat_['song_id'].isin(recomend)][['songName', 'artistName', 'genre', 'release', 'weather','likes', 'region', 'song_id', 'month', 'day', 'release_before', 'air_date','weather_clear','weather_cloud','weather_fog','weather_rain','weather_snow','weather_thunder','weather_disaster']].drop_duplicates()
    return reco 

############################ 22.01.24 수정(노래확인함수) ###########################################

def get_song_info(df, song_id_list):
  program_check=[]
  for id in song_id_list:
    program_check.append((df[(df['song_id'] == id)][['songName', 'artistName' ,'genre', 'release']].drop_duplicates()))
  df_program_check=pd.concat(program_check,ignore_index=True)

  return df_program_check

## 모델링 함수

### 전처리

In [ ]:
def preprocess(df, songDB, tbn):
    tbn_program = tbn.copy()
    #제목과 가수 전처리
    df['songName'] = preprocess_song(df['songName'])   
    df['artistName'] = preprocess_song(df['artistName'])    # 30 second

    df = df.dropna(how='any',axis=0)

    #songdb와 merge
    result = pd.merge(df, songDB, on = ['songName', 'artistName']  )
    #년도 전처리
    result = preprocess_year(result)
    #방송 시작일 전처리
    result = preprocess_airdate(result)       # 100 second
    #장르 전처리
    result = preprocess_genre(result)
    # 프로그램별 빈도수 구하기
    dat_ = data_rating(result)
    # MinMaxScaler를 통해 Scaling   
    dat_ = minmax_scaler(dat_)                # 200 second
    #song_id 별 likes sacling
    dat_ = likes_scaling(dat_)

    #프로그램 label encoding
    le = LabelEncoder()
    dat_['ProgramName']=le.fit_transform(dat_['ProgramName'])
    programs_id = []
    le.classes_

    #tbn의 Program_id 저장
    inverse_id = list(le.inverse_transform(list(dat_['ProgramName'].unique())))
    for i in range(len(inverse_id)):
        if inverse_id[i] in list(tbn_program['ProgramName'].unique()):
            programs_id.append(list(dat_['ProgramName'].unique())[i])

    #빈도수와 likes 수를 점수화 시키키
    dat_ =  rat_like(dat_)  
    dat_ = preprocess_weather(dat_)
    ### 탐색적으로 다양한 추천 후보군 생성
    ##다양한 Meta 후보군 생성
    # dat_['avg_temp'] = dat_['평균기온'].apply(lambda x: temp_bucketize(x))
    dat_['avg_temp'] = dat_.apply(temp_bucketize, axis = 1)

    dat_.loc[dat_['강수량(mm)'] == '-', '강수량(mm)'] = 0
    dat_['강수량(mm)'] = dat_['강수량(mm)'].apply(lambda x : float(x))
    dat_['rain'] = dat_['강수량(mm)'].apply(lambda x: rain_bucketize(x))

    dat_['temp_diff'] = dat_['최고기온'] -  dat_['최저기온']                             
    dat_['temp_diff'] = dat_['temp_diff'].apply(lambda x: temp_diff_bucketize(x))


    ##train, test데이터 분리
    train_df, test_df = train_test_split(dat_)
    train_final = train_df.copy()
    test_final = test_df.copy()

    train_df = train_df.reset_index(drop = True)
    test_df = test_df.reset_index(drop = True)

    ## Feature Mapping - Profiling 피처맵 맵핑
    train_df = featre_mapping(dat_, train_df)
    test_df = featre_mapping(dat_, test_df)

    col_accum_index_dict = col_dict(train_df)
    return dat_, programs_id, le, train_df, test_df, col_accum_index_dict, train_final, test_final

### func_fm

In [ ]:
# class FM(tf.keras.Model):
#     def __init__(self):
#         super(FM, self).__init__()

#         # 모델의 파라미터 정의
#         self.w_0 = tf.Variable([0.0])
#         self.w = tf.Variable(tf.zeros([p]))
#         self.V = tf.Variable(tf.random.normal(shape=(p, k)))

#     def call(self, inputs):
#         linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

#         interactions = 0.5 * tf.reduce_sum(
#             tf.math.pow(tf.matmul(inputs, self.V), 2)
#             - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
#             1,
#             keepdims=False
#         )

#         y_hat = self.w_0 + linear_terms + interactions
#         # return y_hat
#         return y_hat, self.w, self.V

        
def train_on_batch(model, optimizer, accuracy, inputs, targets):
    with tf.GradientTape() as tape:
        y_pred,w,V = model(inputs)
        # loss = tf.keras.losses.binary_crossentropy(from_logits=False,
        #                                            y_true=targets,
        #                                            y_pred=y_pred)
        loss = tf.keras.losses.MSE(y_true=targets,y_pred=y_pred)

        # print(w)
        # print(V)
####################
        error = tf.keras.losses.MSE(y_true=targets,y_pred=y_pred)
        regularizer_w = tf.nn.l2_loss(w)
        regularizer_v = tf.nn.l2_loss(V)
        loss = (error + 0.001 * regularizer_w + 0.001 * regularizer_v)

#####################

        # lambda_w = tf.constant(0.001, name='lambda_w')
        # lambda_v = tf.constant(0.001, name='lambda_v')
        

        # l2_norm = (tf.multiply(lambda_w, tf.pow(w, 2))+
        #                 tf.multiply(lambda_v, tf.pow(V, 2)))
        # print(l2_norm)

        # l2_norm = (tf.math.scalar_mul(0.001, tf.pow(w, 2))+
        #                 tf.math.scalar_mul(0.001, tf.pow(V, 2)))

        # l2_norm = (tf.reduce_sum(
        #             tf.add(
        #                 tf.multiply(lambda_w, tf.pow(w, 2)),
        #                 tf.multiply(lambda_v, tf.pow(V, 2)))))
        # loss = tf.add(error, l2_norm)

    
    # loss를 모델의 파라미터로 편미분하여 gradients를 구한다.
    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용한다.
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # accuracy: update할 때마다 정확도는 누적되어 계산된다.
    accuracy.update_state(targets, y_pred)
    
    return loss


# 반복 학습 함수
def train(X_train, X_test, Y_train, Y_test, p, k, epochs, batch=16, learning_rate=0.001, threshold=0.5):
#    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)
    # X_train, X_test, Y_train, Y_test = xtrain, xtest, ytrain, ytest

    train_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_train, tf.float32), tf.cast(Y_train, tf.float32))).shuffle(5000).batch(batch)

    test_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_test, tf.float32), tf.cast(Y_test, tf.float32))).shuffle(2000).batch(batch)

    print('data transformaiton completed')

    model = FM(p, k)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    accuracy = RootMeanSquaredError()
    loss_history = []

    #checkpoint 생성
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer = optimizer, model = model)
    manager = tf.train.CheckpointManager(ckpt, '/content/drive/MyDrive/temp/fm_check', max_to_keep=3)
    
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
      print("Restored from {}".format(manager.latest_checkpoint))
    else:
      print("Initializing from scratch.")
      
    for i in range(epochs):
      for x, y in train_ds:
          loss = train_on_batch(model, optimizer, accuracy, x, y)
          loss_history.append(loss)
      ckpt.step.assign_add(1)
      if int(ckpt.step) % 1 == 0:
        save_path = manager.save()
        print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      # status.assert_consumed()
      # checkpoint.save(checkpoint_prefix)

      # print(accuracy)

      if i % 1== 0:
          # print("스텝 {:03d}에서 누적 평균 손실: {:.4f} 정확도: {:.4f}".format(i, np.mean(loss_history), accuracy.result().numpy()))
          print("스텝 {:03d}에서 RMSE: {:.4f}".format(i, accuracy.result().numpy()))

          # print("스텝 {:03d}에서 누적 정확도: {:.4f}".format(i, accuracy.result().numpy()))


    test_accuracy = RootMeanSquaredError()

    y_pred_list = []
    for x, y in test_ds:
        y_pred,w,V = model(x)
#        print(y_pred)
        test_accuracy.update_state(y, y_pred)
        y_pred_list.append(y_pred)

    print("테스트 RMSE: {:.4f}".format(test_accuracy.result().numpy()))
    return y_pred_list, model

In [ ]:
class FM(tf.keras.Model):
    def __init__(self, p, k):
        super(FM, self).__init__()

        # 모델의 파라미터 정의
        self.w_0 = tf.Variable([0.0])
        self.w = tf.Variable(tf.zeros([p]))
        self.V = tf.Variable(tf.random.normal(shape=(p, k)))

    def call(self, inputs):
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

        interactions = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(inputs, self.V), 2)
            - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
            1,
            keepdims=False
        )

        y_hat = self.w_0 + linear_terms + interactions
        # return y_hat
        return y_hat, self.w, self.V

In [ ]:
### 모델링

def func_fm(train_df, test_df):
  # GPU 확인
  tf.config.list_physical_devices('GPU')

  # 자료형 선언
  tf.keras.backend.set_floatx('float32')

  # 데이터 로드
  scaler = MinMaxScaler()
  xtrain = train_df.drop(['rating', 'y','likes'], axis = 1)
  ytrain = train_df.loc[:,'rating']
  xtest = test_df.drop(['rating', 'y', 'likes'], axis = 1)
  ytest = test_df.loc[:,'rating']

  xtrain = scaler.fit_transform(xtrain)
  xtest = scaler.transform(xtest)

  n = xtrain.shape[0] + xtest.shape[0]
  p = xtrain.shape[1]  # 예측 변수의 개수
  k = 12  # 잠재 변수의 개수
  batch_size = 16
  epochs = 10

  # tf.debugging.set_log_device_placement(True)
  # 텐서를 GPU에 할당
  with tf.device('/GPU:0'):
    fm_model = train(xtrain, xtest, ytrain, ytest, p, k, 5, batch=512)

  # fm_model[1].save_weights('/content/drive/MyDrive/temp/fm_check')
  return fm_model, scaler

In [ ]:
# fm_model.load_weights('/content/drive/MyDrive/temp/fm_check')

In [ ]:
# model[1].save('/content/drive/MyDrive/temp/mymodel')
# tf.keras.models.load_model('/content/drive/MyDrive/temp/mymodel')

###1. func1 : 연관노래 k개 

In [ ]:
def func1_song_similar(train_final, dat_):
  # #train_final, test_final이용
  train_rate_list = train_final.groupby('ProgramName')[['song_id', 'rating']].apply(lambda x: get_music_list_sort_by_rating(x))

  ##word2vec학습
  ##song_id별 발매일과 장르를 dict로 만들어줌
  song_meta_dict = train_final.set_index('song_id')[['release', 'genre']].to_dict()
  song2vec_dataset = []
  ##song_id, 발매일, 장르를 word2vec을 학습하기 위한 모형으로 만들어줌
  for song_list in train_rate_list:
      meta_list = []
      for music_id in song_list:
          word_meta_1 = "song_id:" + str(music_id)
          word_meta_2 = "year:" + str(song_meta_dict['release'][music_id])
          word_meta_3 = "genre:" + str(song_meta_dict['genre'][music_id])
          meta_list.append(word_meta_1)
          meta_list.append(word_meta_2)
          meta_list.append(word_meta_3)

      song2vec_dataset.append(meta_list)
      
  ##word2vec학습 -> 곡별 유사도를 구할 수 있음.
  model = Word2Vec(song2vec_dataset,
                    window=6,  # 주변 word의 윈도우
                    sg=1,  # skip-gram OR cbow
                    hs=0,  # hierarchical softmax OR negative sampling
                    negative=20,  # negative sampling 파라미터
                    min_count=1  # word의 등장 최소 횟수
                  )

  # #Embedding - 일부 데이터로 song2Vec 학습 결과 확인
  #최근 틀었던 노래 추출
  recent_program_song_list = train_final.groupby('ProgramName')[['song_id', 'air_date']].apply(lambda x: get_recent_song_list_sort_by_time(x, 10))
  # 노래별 연관 노래 k개씩 추출
  k = 20
  #not_in_count = 0
  item2item = {}
  song_unique=dat_['song_id'].unique()
  for song_id in song_unique:                 
      item2item[song_id] = []
      try:
          #곡과 유사한 곡들의 리스트를 추출
          sim_list = model.wv.most_similar("song_id:" + str(song_id), topn=k+20)
          for song_tup in sim_list:
              tup_info = song_tup[0].split(":")
              if (tup_info[0] == "song_id") and (len(item2item[song_id]) < 20):
                  item2item[song_id].append(tup_info[1])
              if (len(item2item[song_id]) >= 20):
                break
      except:
          pass
          #not_in_count += 1
          #print("word", str(song_id) ,"not in vocabulary")

  recent_song_similar_items = recent_program_song_list.apply(lambda x: get_similar_items(item2item, x, k=30))

  return recent_song_similar_items

###2. func2 : 인기있는 곡 

In [ ]:
def func2_song_rating(dat_, train_final):
  mean_rating = train_final.groupby('song_id')['rating'].agg(['mean', 'count'])

  ###빈도수의 평균이 높은 상위 30개 노래
  popular_song_list = mean_rating[mean_rating['count']>5]['mean'].sort_values(ascending=False).index[:30].tolist()

  ##장르, 연도, 날씨를 meta로 하여, meta별 빈도수 평균이 높은 상위 30개의 노래
  merge_df = pd.merge(mean_rating, dat_, on='song_id')
  genre_popular = merge_df.groupby('genre').apply(lambda x: get_meta_popular_list_candidate(x, k=50))
  year_popular = merge_df.groupby('release').apply(lambda x: get_meta_popular_list_candidate(x, k=50))
  weather_popular = get_meta_popular_list_candidate_weather(merge_df, k=50)

  train_year_list = train_final.groupby(by=['ProgramName','release'], as_index=False)['song_id'].count()
  train_year_list=train_year_list.loc[train_year_list.groupby(['ProgramName'])['song_id'].idxmax()]


  train_genre_list=train_final.groupby(by=['ProgramName','genre'], as_index=False)['song_id'].count()
  train_genre_list=train_genre_list.loc[train_genre_list.groupby(['ProgramName'])['song_id'].idxmax()]

  tf_weather=train_final.loc[:,['ProgramName','weather_clear','weather_cloud','weather_fog','weather_rain','weather_snow','weather_thunder','weather_disaster']]
  train_weather_list=tf_weather.groupby(by=['ProgramName'], as_index=False).agg(['sum'])

  weather_list=[]

  for idx,count_row in train_weather_list.iterrows():
    max=-1
    idx_check=-1
    for count_num,temp_idx in zip(count_row,range(len(count_row))) :
      if max<count_num:
        max=count_num
        idx_check=temp_idx
    weather_list.append(idx_check)

  recommendations = dict()

  for idx,[a,year],[b,genre],wt in zip(range(len(train_year_list)),train_year_list.iterrows(),train_genre_list.iterrows(),weather_list):
    recommendations[idx]= list(set(year_popular[year[1]] + genre_popular[genre[1]]+weather_popular[wt]))

  return recommendations

In [ ]:
def combine_func1_func2(train_final, dat_):
  #train_final, test_final이용
  # train_rate_list = train_final.groupby('ProgramName')[['song_id', 'rating']].apply(lambda x: get_music_list_sort_by_rating(x))
  recent_song_similar_items = func1_song_similar(train_final, dat_)
  recommendations = func2_song_rating(dat_, train_final)

  return recent_song_similar_items, recommendations

### func_pred

In [ ]:
def pronumtoname(x, le):
    return ("".join(le.inverse_transform([x])[0].split()))

def make_songpool_for_eachpro(pro_id, train_final, test_final, train_df, test_df):
    # 사람용 song pool
    song_book_all= pd.concat([train_final, test_final], axis=0, ignore_index=True)
    song_book = song_book_all[['song_id','songName','artistName','album','release','genre']]
    song_book = song_book.drop_duplicates(subset=['song_id'])

    # song pool
    song_pool_all= pd.concat([train_df, test_df], axis=0, ignore_index=True)
    song_pool = song_pool_all[['genre','release','song_id']]
    song_pool = song_pool.drop_duplicates(subset=['song_id'])

    # sampling weather conditions for a test
    df_all = pd.concat([train_df, test_df], axis=0)
    test_pred_d = df_all.sample(3)
    test_pred_days = test_pred_d[['weather_clear', 'weather_cloud',
          'weather_fog', 'weather_rain', 'weather_snow', 'weather_thunder',
          'weather_disaster','avg_temp','rain','temp_diff','month','day']]
    p_id = pro_id
    ProgramName = pd.DataFrame({'ProgramName':[p_id, p_id, p_id]})
    test_pred_days['ProgramName'] = [p_id, p_id, p_id]

    colname_days = ['ProgramName','weather_clear', 'weather_cloud',
          'weather_fog', 'weather_rain', 'weather_snow', 'weather_thunder',
          'weather_disaster','avg_temp','rain','temp_diff','month','day']
    test_pred_days = test_pred_days[colname_days]

    # test_pred = test_pred_days.merge(song_pool, how='cross') #python version 1.2> needed
    test_pred_days['key'] = 1
    song_pool['key'] = 1
    test_pred = pd.merge(test_pred_days, song_pool, on ='key').drop("key", 1)

    colname = ['ProgramName', 'weather_clear', 'weather_cloud',
          'weather_fog', 'weather_rain', 'weather_snow', 'weather_thunder',
          'weather_disaster', 'avg_temp', 'genre',
           'release', 'song_id', 'rain', 'temp_diff', 'month', 'day']
    test_pred = test_pred[colname]
    return test_pred, song_book

def make_pred_list(batch, pro_id, model, test_pred, song_book, scaler, le):
    # test_pred_scaled = test_pred.astype('float')
    test_pred_scaled = scaler.transform(test_pred)
    batch = batch
    test_pred_tensor = tf.data.Dataset.from_tensor_slices(
            (tf.cast(test_pred_scaled, tf.float32))).batch(batch)

    rslt_pred = []
    for x in test_pred_tensor:
    #     pred = model(x)
        # rslt_pred.append(model(x))
        rslt_pred.append(model(x)[0])
    rslt = sum([i.numpy().tolist() for i in rslt_pred], [])
    # rslt = sum(rslt_pred, [])

    test_pred['pred'] = rslt

    test_pred_dropped = test_pred.drop(['genre','release'], axis=1)
    test_pred_merged = pd.merge(test_pred_dropped, song_book, left_on='song_id', right_on='song_id', how='left')

    p_id = pro_id
    test_pred_merged['Program']=pronumtoname(p_id, le)
    # test_pred_merged['Program']=test_pred_merged['ProgramName'].apply(pronumtoname)

    return test_pred_merged

In [ ]:
# day별로 나누기
def day_by_day(test_pred_merged):
  len1 = (len(test_pred_merged)/3 - 1)
  len2 = (len(test_pred_merged)*2/3 - 1)

  day1 = test_pred_merged.loc[:len1]
  day2 = test_pred_merged.loc[(len1+1):len2]
  day3 = test_pred_merged.loc[(len2+1):]
  return day1, day2, day3

## 메인 함수

In [ ]:
def main_func(user_df, songDB, tbn, p_id, batch_size, w1, w2):
  dat_, programs_id, le, train_df, test_df, col_accum_index_dict, train_final, test_final = preprocess(user_df, songDB, tbn) #전처리
  fm_model = func_fm(train_df, test_df)
  recent_song_similar_items, recommendations = combine_func1_func2(train_final, dat_) #func1, func2 실행 후 합침
  # p_id = 256
  # batch_size = 512

  scaler = fm_model[1]
  test_pred, song_book = make_songpool_for_eachpro(p_id, train_final, test_final, train_df, test_df)
  results_pro_pid = make_pred_list(batch_size, p_id, fm_model[0][1], test_pred, song_book, scaler, le)

  df_similar = pd.DataFrame({'song_id':[int(x) for x in recent_song_similar_items[p_id]], 'similar':1})
  df_popular = pd.DataFrame({'song_id':[int(x) for x in recommendations[p_id]], 'popular':1})
  # df_merged = pd.merge(day1, df_similar, how='left', on='song_id')
  df_merged = pd.merge(results_pro_pid, df_similar, how='left', on='song_id')
  df_merged = pd.merge(df_merged, df_popular, how='left', on='song_id')
  df_merged['similar'] = df_merged['similar'].fillna(0)
  df_merged['popular'] = df_merged['popular'].fillna(0)

  # w1 = 0.4
  # w2 = 0.2
  df_merged['pred_fin'] = w1 * df_merged['similar'] + w2 * df_merged['popular'] + df_merged['pred']
  return df_merged

In [ ]:
df_final = main_func(user_df, songDB, tbn, 256, 512, 0.4, 0.2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:541: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:542: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:548: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

data transformaiton completed
Restored from /content/drive/MyDrive/temp/fm_check/ckpt-62
Saved checkpoint for step 64: /content/drive/MyDrive/temp/fm_check/ckpt-63
스텝 000에서 RMSE: 0.9651
Saved checkpoint for step 65: /content/drive/MyDrive/temp/fm_check/ckpt-64
스텝 001에서 RMSE: 0.9650
Saved checkpoint for step 66: /content/drive/MyDrive/temp/fm_check/ckpt-65
스텝 002에서 RMSE: 0.9651
Saved checkpoint for step 67: /content/drive/MyDrive/temp/fm_check/ckpt-66
스텝 003에서 RMSE: 0.9651
Saved checkpoint for step 68: /content/drive/MyDrive/temp/fm_check/ckpt-67
스텝 004에서 RMSE: 0.9651
테스트 RMSE: 0.9656


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 완성된 결과 적용(추천 실행)

In [ ]:
day1, day2, day3 = day_by_day(df_final)

In [ ]:
day1[['Program', 'songName', 'artistName', 'release', 'genre', 'pred_fin']].sort_values(by='pred_fin', ascending=False).head(30)

,Program,songName,artistName,release,genre,pred_fin
1730,한밤의교차로,세월이 가면,최호섭,1985,Ballade,3.065925
1731,한밤의교차로,세월이 가면,최호섭,1985,Ballade,3.065925
1160,한밤의교차로,하늘을 달리다,이적,2000,Rock,3.046499
1547,한밤의교차로,bravo my life,봄여름가을겨울,2000,Rock,3.045370
1548,한밤의교차로,bravo my life,봄여름가을겨울,2000,Rock,3.045370
970,한밤의교차로,고마워요,임현정,2000,Rock,3.043594
1865,한밤의교차로,어디쯤 가고 있을까,전영,2000,Folk,3.027205
1864,한밤의교차로,어디쯤 가고 있을까,전영,2000,Folk,3.027205
5896,한밤의교차로,터,재주소년,2020,Ballade,3.007779
12433,한밤의교차로,님,김정호,2000,adult,2.890665
